In [1]:
from dynamicSceneGenerator import DynamicSceneGenerator
from visualize import visualize

In [2]:
# Generate dynamic scene with random tracks
dsg = DynamicSceneGenerator()
dsg.set_random_vessels(6)
dsg.generate_random_tracks()
vessels = dsg.get_vessels()
visualize(vessels)


![SegmentLocal](dynamicSceneExample.gif "Dynamic scene example")